# Demo While Activity with ShiftAmount Activity
The combination of a while activity with a shift amount activity can be used to represent the loading or unloading of a vessel, where coordination of the process is represented on the granularity of the amount shifted in one iteration.

In [1]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of Sites

In [8]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5b",
    "geometry": location_from_site,
    "capacity": 10,
    "level": 10,
}

location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)

data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5b",
    "geometry": location_to_site,
    "capacity": 10,
    "level": 0,
}

from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

## Creation of Vessel

In [15]:
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

def compute_v_provider(v_empty, v_full):
    return lambda x: 10

# TSHD variables
data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",
    "geometry": location_from_site,
    "loading_rate": 1,
    "unloading_rate": 1,
    "capacity": 5,
    "compute_v": compute_v_provider(5, 4.5),
}

hopper = TransportProcessingResource(**data_hopper)

## Definition of Shift Amount activity

In [10]:
shift_amount_activity_loading_data = {
    "env": my_env,
    "name": "Transfer MP",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff52",
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 1,
    "duration": 20,
    "postpone_start": True,
}
activity = model.ShiftAmountActivity(**shift_amount_activity_loading_data)

while_data = {
    "env": my_env,
    "name": "while",
    "registry": registry,
    "sub_process": activity,
    "condition_event": [{"type":"container", "concept": hopper, "state":"full"}],
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


## Run simulation

In [11]:
my_env.run()


In [12]:
log_df = pd.DataFrame(while_activity.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,conditional process while,START,1970-01-01 00:00:40,-1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
1,sub process Transfer MP,START,1970-01-01 00:00:40,-1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
2,Transfer MP,START,1970-01-01 00:00:40,1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
3,Transfer MP,STOP,1970-01-01 00:01:00,1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
4,sub process Transfer MP,STOP,1970-01-01 00:01:00,-1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
5,sub process Transfer MP,START,1970-01-01 00:01:00,-1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
6,Transfer MP,START,1970-01-01 00:01:00,1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
7,Transfer MP,STOP,1970-01-01 00:01:20,1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
8,sub process Transfer MP,STOP,1970-01-01 00:01:20,-1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
9,sub process Transfer MP,START,1970-01-01 00:01:20,-1.0,6dbbbdf7-4589-11e9-bf3b-b469212bff5g


The resulting levels of objects in the hopper and the from_site are requested below. 

In [13]:
hopper.container.get_level()

5.0

In [14]:
from_site.container.get_level()

5.0